<div style="text-align: center; font-size: 40px; font-weight: bold;">
    Inverted Pendulum (Reaction Wheel Model)
</div>

<div style="text-align: center;">
    <img src="diagram.jpg" alt="Diagram" style="width:30%; height:auto;">
</div>

<div style="text-align: center;">
    <a href="diagram.pdf" target="_blank">View PDF</a>
</div>

<div style="display: flex; justify-content: center;">

| Variable   | Definition                                                       | Units          |
|------------|------------------------------------------------------------------|----------------|
| $m$        | Total mass of the system                                         | $kg$           |
| $l$        | Length from reaction wheel centre to pivoting point              | $m$            |
| $I_P$      | Moment of inertia of the entire system about the pivoting point  | $kg \cdot m^2$ |
| $I_F$      | Moment of inertia of the reaction wheel about its centre         | $kg \cdot m^2$ |
| $\theta$   | Angle from the upright position about the pivoting point         | $radian$       |
| $\phi$     | Angle of the reaction wheel about its centre                     | $radian$       |

</div>

## Lagrangian Derivation:

> Total kinetic energy of the system (both the pendulum and reaction wheel, where the reaction wheel rotates with angular velocity [$\dot{\theta} + \dot{\phi}$])

\begin{aligned}
T &= \frac{1}{2} \cdot \left( I_P + m \cdot {l^2} \right) {\dot \theta}^2 + I_F \left({\dot \theta} + {\dot \phi}\right)^2
\end{aligned}

> Total Potential Energy of the system:

\begin{aligned}
V &= -mgl \cdot \cos(\theta)
\end{aligned}

> The Lagrangian is defined as:

\begin{aligned}
\mathcal{L} &= T - V \\
\mathcal{L} &= \frac{1}{2} \cdot \left( I_P + m {l^2} \right) {\dot \theta}^2 + I_F \left({\dot \theta} + {\dot \phi}\right)^2 + mgl \cdot \cos(\theta)
\end{aligned}


## Differential Equations:

\begin{aligned}
\left( I_P + m {l^2} + I_F \right) {\ddot \theta} + {I_F {\ddot \phi}} + {mgl \cdot \cos(\theta)} &= 0 \quad (1) \\
{I_F} \left({\ddot \theta} + {\ddot \phi} \right) &= u \quad (2)
\end{aligned}

> Note: Equation (2) follows the general form of rotational dynamics: $I \cdot \alpha = \tau$

> Our control input $u$ will be $\tau$ (torque generated by a BLDC motor, applied to the reaction wheel).

## Linearisation about the Upright Equilibrium
> For small angular deviations ($\theta$ is small, so $\sin({\theta}) {\approx} {\theta}$) the equations linearise. 

- Solving equation $(2)$ for $\ddot{\phi}$:

\begin{aligned}
{\ddot{\phi}} &= \frac{u}{I_F} - \ddot \theta \\
\end{aligned}

- Substitute into equation $(1)$:

\begin{aligned}
\left( I_P + m l^2 + I_F \right) \ddot{\theta} + I_F \left( \frac{u}{I_F} - \ddot{\theta} \right) + m g l \theta = 0
\end{aligned}


- Collect the $\ddot{\theta}$ terms:

\begin{aligned}
\Bigl[ \left( I_P + m l^2 + I_F \right) - I_F \Bigr] \ddot{\theta} + mgl\,\theta + u = 0
\end{aligned}

- $I_F$ cancels out:

\begin{aligned}
\left( I_P + m l^2 \right)\ddot{\theta} + mgl\,\theta + u = 0
\end{aligned}

##### The linearised equations become:
> Pendulum (angular) dynamics:

\begin{aligned}
\ddot{\theta} = -\frac{mgl}{I_P+ml^2}\,\theta - \frac{1}{I_P+ml^2}\,u
\end{aligned}

> Reaction wheel (angular) dynamics:

\begin{aligned}
\ddot{\phi} = \frac{u}{I_P} - \ddot{\theta} = \frac{u}{I_P} + \frac{mgl}{I_P+ml^2}\,\theta + \frac{1}{I_P+ml^2}\,u
\end{aligned}

## Defining the State Variable

\begin{aligned}
x &= \begin{bmatrix}
x_1 = \theta \\
x_2 = \dot{\theta} \\
x_3 = \phi \\
x_4 = \dot{\phi}
\end{bmatrix} \\
\end{aligned}

>- [$\theta$] describes the angle of the pendulum arm
>- [$\phi$] describes the angle of the reaction wheel

## State Equations:

\begin{aligned}
\dot{x} = \begin{bmatrix}
\dot{x}_1 = \dot{\theta} &= x_2 \\
\dot{x}_2 = \ddot{\theta} &= \frac{-mgl}{I_P + m l^2} x_1 - \frac{1}{I_P + m l^2}u \\
\dot{x}_3 = \dot{\phi} &= \dot{x}_4 \\
\dot{x}_4 = \ddot{\phi} &= \frac{mgl}{I_P + m l^2} x_1 + \left( \frac{1}{I_F} + \frac{1}{I_P + m l^2} \right)u
\end{bmatrix}
\end{aligned}

## System in state-space form:

\begin{aligned}
\dot{x} = A{x} + B{u}
\end{aligned}


\begin{aligned}
A = \begin{bmatrix}
0 & 1 & 0 & 0 \\
\frac{-mgl}{I_P + m l^2} & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
\frac{mgl}{I_P + m l^2} & 0 & 0 & 0
\end{bmatrix} \quad
\end{aligned}


\begin{aligned}
B = \begin{bmatrix}
0 \\
\frac{-1}{I_P + m l^2} \\
0 \\
\frac{1}{I_F} + \frac{1}{I_P + m l^2}
\end{bmatrix}
\end{aligned}


## Test:

> Note: Variables are arbitrary for now

We will define an A and B matrix and check if the system is controllable. If we have full row rank (each of the rows of the matrix are linearly independent), then the system is controllable.

In [2]:
import numpy as np

# Define the parameters (replace with your actual values)
m = 1  # mass of the pendulum bob
l = 1  # length of the pendulum arm
g = 9.81  # acceleration due to gravity
Ip = 1  # moment of inertia of the pendulum
If = 1  # moment of inertia of the reaction wheel

# Define matrix A
A = np.array([
    [0, 1, 0, 0],
    [-m * g * l / (Ip + m * l**2), 0, 0, 0],
    [0, 0, 0, 1],
    [m * g * l / (Ip + m * l**2), 0, 0, 0]
])

# Define matrix B
B = np.array([
    [0],
    [-1 / (Ip + m * l**2)],
    [0],
    [1 / If + 1 / (Ip + m * l**2)]
])

# Compute the controllability matrix
n_states = A.shape[0]  # Number of states
controllability_matrix = B
for i in range(1, n_states):
    controllability_matrix = np.hstack((controllability_matrix, np.linalg.matrix_power(A, i) @ B))

# Check the rank of the controllability matrix
rank = np.linalg.matrix_rank(controllability_matrix)

print("Matrix A:\n", A)
print("\nMatrix B:\n", B)
print("\nControllability Matrix:\n", controllability_matrix)
print("\nRank of Controllability Matrix:", rank)

# Check if the system is controllable
if rank == n_states:
    print("\nThe system is controllable.")
else:
    print("\nThe system is not controllable.")


Matrix A:
 [[ 0.     1.     0.     0.   ]
 [-4.905  0.     0.     0.   ]
 [ 0.     0.     0.     1.   ]
 [ 4.905  0.     0.     0.   ]]

Matrix B:
 [[ 0. ]
 [-0.5]
 [ 0. ]
 [ 1.5]]

Controllability Matrix:
 [[ 0.     -0.5     0.      2.4525]
 [-0.5     0.      2.4525  0.    ]
 [ 0.      1.5     0.     -2.4525]
 [ 1.5     0.     -2.4525  0.    ]]

Rank of Controllability Matrix: 4

The system is controllable.
